In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [ ]:
## Read Data for the Fraudulent Email Kaggle Challenge
import pandas as pd
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

### Let's divide the training and test set into two partitions

In [ ]:
from sklearn.model_selection import train_test_split
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(data["text"])
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, data["label"], test_size=0.2, random_state=42)

## Data Preprocessing

In [ ]:
import nltk
nltk.download('all')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import re
from nltk.corpus import stopwords
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\<.*?\>', '', text) 
    text = re.sub(r'[^a-zA-Z]', ' ', text)  
    text = ' '.join([word for word in text.split() if word not in stop_words])  
    return text

data["text"] = data["text"].apply(preprocess_text)

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
def clean_html(html_text):
     
    soup = BeautifulSoup(html_text, "html.parser")
    
    
    for script in soup(["script", "style"]):
        script.decompose()  
        
    
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()  
        
    cleaned_text = soup.get_text()  
    
    
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  
    cleaned_text = cleaned_text.strip()  
    
    return cleaned_text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r"\bb\b", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\b\w\b", "", text)
    text = re.sub(r"^\w\s", "", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
def remove_stopwords(text):
    
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)
    
    return cleaned_text

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
def get_top_words(df, label, n=10):
    text = ' '.join(df[df['label'] == label]['text'])
    words = text.split()
    counter = Counter(words)
    return counter.most_common(n)

ham_top_words = get_top_words(data, label=0)  
spam_top_words = get_top_words(data, label=1)  

print("Top 10 words in ham messages:", ham_top_words)
print("Top 10 words in spam messages:", spam_top_words)

## Extra features

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(data["text"])
def get_top_tfidf_words(vectorizer, X, n=10):
    feature_array = vectorizer.get_feature_names_out()
    tfidf_sorting = X.toarray().sum(axis=0).argsort()[::-1]
    top_n_words = [(feature_array[i], X.toarray().sum(axis=0)[i]) for i in tfidf_sorting[:n]]
    return top_n_words

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

data = pd.DataFrame({
    'text': ["This is a sample sentence.", "Another example sentence for testing.", "Free money in your bank account.", "Transfer funds to win big prizes!"]
})

# Splitting the data into training and test sets
X_train, X_test = train_test_split(data['text'], test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform both training and test data
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert the result to a DataFrame for easier visualization
X_train_bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow_df = pd.DataFrame(X_test_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the Bag of Words representation
print("Training Data BoW:")
print(X_train_bow_df.head())

print("\nTest Data BoW:")
print(X_test_bow_df.head())

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code


data = pd.DataFrame({
    'text': [
        "This is a sample sentence.",
        "Another example sentence for testing.",
        "Free money in your bank account.",
        "Transfer funds to win big prizes!"
    ]
})

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the entire dataset and transform it into TF-IDF features
tfidf_matrix = vectorizer.fit_transform(data['text'])

# Print the shape of the vectorized dataset
print("Shape of the vectorized dataset:", tfidf_matrix.shape)


## And the Train a Classifier?

In [ ]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(report)

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code